In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.utils import plot_model
from keras import regularizers
from sklearn.metrics import roc_curve, auc

In [ ]:
pho = np.load('photon.npy')
#hal = np.load('beamhalo.npy')
hal = np.load('photon.npy')
print ("photon dataset shape:",pho.shape[0])
print ("halo dataset shape:",hal.shape[0])
#nstrain = min(pho.shape[0],hal.shape[0])
nstrain = 2515
print ("samples from each dataset:",nstrain)

In [ ]:

X_train = np.vstack((pho[:nstrain,:,:],hal[:nstrain,:,:]))
Y_train = np.vstack((keras.utils.to_categorical(np.ones(nstrain),2)  , keras.utils.to_categorical(np.zeros(nstrain),2)))

print (X_train.shape)
print (Y_train.shape)
print Y_train

In [ ]:
X_train=X_train.reshape(X_train.shape[0],11,11,1)
print "creating model=========>"
model = Sequential()
model.add(Conv2D(30, (3, 3),strides=(1, 1), padding="valid", activation="relu", input_shape=(11,11,1),kernel_regularizer=regularizers.l2(0.01) ))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(10, activation='relu'))
model.add(Dense(Y_train.shape[1], activation='softmax'))

#ADAM = keras.optimizers.Adam(lr=500.005)
# Compile model
print "compilation up next=======>"
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['binary_accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='ADAM',metrics=['binary_accuracy'])
model.summary()
#plot_model(model, to_file='modelCNN.png',show_shapes=True,show_layer_names=True)
#fitting
print "fitting now=========>"
history = model.fit(X_train,Y_train , batch_size=20, epochs=200, shuffle=True, validation_split=0.2, verbose=1)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validatoin'], loc='upper left')
plt.show()